In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.91 sec - Loss 0.105701 - ACC 83.54% - ACC Mean 83.54% - AUC 90.79% - AUC Mean 90.79% - Deter 000
Ite 00050 - 0.14 sec - Loss 0.098855 - ACC 83.21% - ACC Mean 83.93% - AUC 90.42% - AUC Mean 90.70% - Deter 042
Ite 00100 - 0.16 sec - Loss 0.081510 - ACC 84.40% - ACC Mean 84.24% - AUC 90.86% - AUC Mean 90.56% - Deter 092
Ite 00150 - 0.23 sec - Loss 0.051500 - ACC 84.82% - ACC Mean 83.64% - AUC 91.07% - AUC Mean 90.68% - Deter 142
Ite 00200 - 0.16 sec - Loss 0.031194 - ACC 84.25% - ACC Mean 84.47% - AUC 92.13% - AUC Mean 91.57% - Deter 000
Ite 00250 - 0.12 sec - Loss 0.024782 - ACC 83.39% - ACC Mean 84.18% - AUC 91.84% - AUC Mean 91.68% - Deter 050
Ite 00300 - 0.15 sec - Loss 0.022782 - ACC 83.41% - ACC Mean 83.96% - AUC 90.89% - AUC Mean 91.47% - Deter 100
Ite 00350 - 0.09 sec - Loss 0.021857 - ACC 87.47% - ACC Mean 84.31% - AUC 92.58% - AUC Mean 91.57% - Deter 000
Ite 00400 - 0.12 sec - Loss 0.021241 - ACC 85.00% - ACC Mean 84.89% - AUC 92.42% - AUC Mean 92.09% - Deter 006
I

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 83.47% +- 0.0% - auc: 90.52% +- 0.0%
MEAN - acc: 85.91% +- 0.0% - auc: 91.2% +- 0.0%
KNN - acc: 88.08% +- 0.0% - auc: 91.53% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:45<00:00, 109.71it/s]



GAIN RUN 1/10 - acc: 86.53% +- 1.27% - auc: 90.97% +- 2.93%



100%|██████████| 5000/5000 [00:45<00:00, 110.55it/s]



GAIN RUN 2/10 - acc: 85.89% +- 2.73% - auc: 90.09% +- 2.92%



100%|██████████| 5000/5000 [00:45<00:00, 110.24it/s]


...


GAIN RUN 8/10 - acc: 84.13% +- 3.33% - auc: 91.05% +- 1.45%



100%|██████████| 5000/5000 [00:37<00:00, 133.83it/s]



GAIN RUN 9/10 - acc: 85.83% +- 2.34% - auc: 91.75% +- 1.44%



100%|██████████| 5000/5000 [00:37<00:00, 132.79it/s]



GAIN RUN 10/10 - acc: 84.92% +- 3.86% - auc: 92.54% +- 1.87%

GAIN GLOBAL - acc: 85.14% +- 0.91% - auc: 91.36% +- 0.73%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 87.98% +- 0.0% - auc: 92.43% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 1/10 - acc: 85.09% +- 5.73% - auc: 89.59% +- 5.09%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8

MISSFOREST RUN 2/10 - acc: 83.68% +- 6.02% - auc: 89.85% +- 3.83%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 3/10 - acc: 87.17% +- 2.73% - auc: 91.73% +- 3.62%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 4/10 - acc: 83.63% +- 4.42% - auc: 90.95% +- 3.82%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 5/10 - acc: 85.83% +- 2.83% - auc: 92.92% +- 1.26%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8

MISSFOREST RUN 6/10 - acc: 82.48% +

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 84.77% +- 6.0% - auc: 91.17% +- 3.65%
SOFTIMPUTE RUN 2/10 - acc: 85.12% +- 4.79% - auc: 91.82% +- 1.55%
SOFTIMPUTE RUN 3/10 - acc: 84.82% +- 6.18% - auc: 91.83% +- 3.87%
SOFTIMPUTE RUN 4/10 - acc: 85.5% +- 5.94% - auc: 91.3% +- 3.96%
SOFTIMPUTE RUN 5/10 - acc: 85.03% +- 6.1% - auc: 90.35% +- 3.57%
SOFTIMPUTE RUN 6/10 - acc: 84.95% +- 4.49% - auc: 91.47% +- 3.57%
SOFTIMPUTE RUN 7/10 - acc: 84.08% +- 1.68% - auc: 91.42% +- 2.31%
SOFTIMPUTE RUN 8/10 - acc: 83.14% +- 7.13% - auc: 89.42% +- 3.96%
SOFTIMPUTE RUN 9/10 - acc: 83.09% +- 5.92% - auc: 90.07% +- 3.65%
SOFTIMPUTE RUN 10/10 - acc: 84.3% +- 5.21% - auc: 92.31% +- 3.77%

SOFTIMPUTE GLOBAL - acc: 84.48% +- 0.78% - auc: 91.12% +- 0.85%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.5535

SINKHORN RUN 1/10 - acc: 86.45% +- 1.4% - auc: 93.5% +- 1.92%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.5059

SINKHORN RUN 2/10 - acc: 86.07% +- 3.49% - auc: 90.42% +- 3.31%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.4620

SINKHORN RUN 3/10 - acc: 88.26% +- 2.7% - auc: 92.83% +- 2.34%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.5249

SINKHORN RUN 4/10 - acc: 85.39% +- 4.17% - auc: 90.6% +- 1.88%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.5598

SINKHORN RUN 5/10 - acc: 87.7% +- 3.65% - auc: 92.28% +- 2.34%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.5161

SINKHORN RUN 6/10 - acc: 89.7% +- 1.86% - auc: 91.61% +- 1.26%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.4920

SINKHORN RUN 7/10 - acc: 85.3% +- 2.39% - auc: 89.97% +- 1.72%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.4799

SINKHORN RUN 8/10 - acc: 85.91% +- 5.22% - auc: 90.97% +

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 84.31% +- 2.68% - auc: 90.35% +- 2.31%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 84.67% +- 3.9% - auc: 92.36% +- 1.38%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 84.86% +- 4.69% - auc: 91.72% +- 2.02%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 86.49% +- 3.72% - auc: 91.28% +- 2.04%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 83.56% +- 4.2% - auc: 92.3% +- 2.8%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 86.94% +- 3.95% - auc: 92.49% +- 1.83%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 85.43% +- 3.24% - auc: 91.36% +- 2.4%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 86.68% +- 3.57% - auc: 91.81% +- 2.27%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 8

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly better than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly worse than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly worse than MICE
Metric AUC - OUR METHOD is even with MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is even with SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly bette